In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=0f103c9de14c8ae2c667b38651c7b58cd5c0e317f4d42d5051b157d9465c857f
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F20 '''
def F20(X):
    f = bn.F20()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 26.081302221157618 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_201000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_20_200Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 50, [5] * 50 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=985778, Mon Apr 13 20:10:21 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 3.533057582971711e+05 1.0e+00 2.37e+00  2e+00  2e+00 0:00.0
    2     30 3.527331155149209e+05 1.0e+00 2.28e+00  2e+00  2e+00 0:00.0
    3     45 2.871822338497468e+05 1.0e+00 2.22e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [465238.19793953] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [434586.19633797] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -5.329513582709217e+05 1.4e+00 2.31e-01  2e-01  2e-01 0:01.8
  200   3000 -5.482967832808137e+05 1.4e+00 1.49e-02  1e-02  1e-02 0:03.5
  300   4500 -5.483625001833913e+05 1.5e+00 8.75e-04  7e-04  8e-04 0:05.2
  400   6000 -5.483626713744077e+05 1.5e+00 6.69e-05  6e-05  6e-05 0:06.9
  500   7500 -5.483626721775878e+05 1.6e+00 4.68e-06  4e-06  4e-06 0:08.7
  600   9000 -5.483626721825477e+05 1.6e+00 3.10e-07  2e-07  3e-07 0:10.4
  700  10500 -5.483626721825661e+05 1.8e+00 3.66e-08  3e-08  3e-08 0:12.1
  800  12000 -5.483626721825664e+05 2.0e+00 2.37e-08  2e-08  2e-08 0:13.9
  900  13500 -5.483626721825671e+05 2.2e+00 1.63e-08  1e-08  1e-08 0:15.6
 1000  15000 -5.483626721825666e+05 2.5e+00 1.14e-08  8e-09  1e-08 0:17.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -5.483626721825668e+05 2.6e+00 8.69e-09  6e-09  7e-09 0:19.1
 1200  18000 -5.483626721825671e+05 2.6e+00 7.95e-09  5e-09  7e-09 0:20.9
 1300  19500 -5.483626721825668e+05 2.7e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [489874.99918622] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=489874.999186, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=489874.999186, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)


  100   1500 -1.115955131956867e+05 1.5e+00 4.12e-01  4e-01  4e-01 0:02.1
  200   3000 -1.276469776530637e+05 1.7e+00 5.00e-02  4e-02  5e-02 0:04.3
  300   4500 -1.279860169364017e+05 2.3e+00 1.33e-02  1e-02  2e-02 0:06.4
  400   6000 -1.279983078870991e+05 2.7e+00 2.24e-03  2e-03  3e-03 0:08.5
  500   7500 -1.279986569266688e+05 3.0e+00 3.08e-04  2e-04  5e-04 0:10.7
  600   9000 -1.279986641247730e+05 3.4e+00 4.78e-05  3e-05  8e-05 0:12.9
  700  10500 -1.279986642495387e+05 3.7e+00 1.03e-05  6e-06  2e-05 0:15.0
  800  12000 -1.279986642606654e+05 5.7e+00 5.93e-06  3e-06  2e-05 0:17.1
  900  13500 -1.279986642649271e+05 1.0e+01 1.67e-06  8e-07  7e-06 0:19.3
 1000  15000 -1.279986642650599e+05 1.1e+01 1.56e-07  8e-08  7e-07 0:21.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.279986642650606e+05 1.1e+01 2.81e-08  1e-08  1e-07 0:23.5
 1200  18000 -1.279986642650607e+05 1.1e+01 1.44e-08  7e-09  6e-08 0:25.7
 1300  19500 -1.279986642650606e+05 1.2e

...

  100   1500 -1.026302754481981e+05 1.5e+00 4.79e-01  4e-01  5e-01 0:02.1
  200   3000 -1.238152083584470e+05 1.8e+00 8.80e-02  7e-02  1e-01 0:04.2
  300   4500 -1.249660825887722e+05 2.6e+00 5.92e-02  5e-02  9e-02 0:06.3
  400   6000 -1.277691295944524e+05 4.9e+00 5.26e-02  4e-02  1e-01 0:08.4
  500   7500 -1.279944845682825e+05 4.5e+00 8.40e-03  6e-03  2e-02 0:10.5
  600   9000 -1.279983299362955e+05 4.2e+00 2.16e-03  1e-03  4e-03 0:12.7
  700  10500 -1.279986336531653e+05 4.5e+00 4.94e-04  3e-04  1e-03 0:14.8
  800  12000 -1.279986624031545e+05 6.7e+00 1.65e-04  9e-05  5e-04 0:16.9
  900  13500 -1.279986641752354e+05 9.7e+00 3.66e-05  2e-05  2e-04 0:19.1
 1000  15000 -1.279986642638524e+05 1.0e+01 5.73e-06  3e-06  2e-05 0:21.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.279986642650504e+05 1.1e+01 5.97e-07  3e-07  2e-06 0:23.3
 1200  18000 -1.279986642650606e+05 1.1e+01 6.73e-08  3e-08  3e-07 0:25.4
 1300  19500 -1.279986642650606e+05 1.2e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [609335.22428148] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [524630.71478285] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -5.165602572914980e+05 1.5e+00 3.52e-01  3e-01  4e-01 0:02.0
  200   3000 -5.482669506612268e+05 1.5e+00 2.16e-02  2e-02  2e-02 0:04.0
  300   4500 -5.483624130520866e+05 1.6e+00 1.05e-03  9e-04  1e-03 0:06.0
  400   6000 -5.483626704212852e+05 1.6e+00 7.75e-05  6e-05  7e-05 0:08.0
  500   7500 -5.483626721771063e+05 1.6e+00 4.44e-06  3e-06  4e-06 0:10.1
  600   9000 -5.483626721825477e+05 1.6e+00 2.95e-07  2e-07  3e-07 0:12.1
  700  10500 -5.483626721825668e+05 1.6e+00 4.09e-08  3e-08  3e-08 0:14.1
  800  12000 -5.483626721825666e+05 1.8e+00 1.70e-08  1e-08  1e-08 0:16.1
  900  13500 -5.483626721825666e+05 2.0e+00 1.60e-08  1e-08  1e-08 0:18.1
 1000  15000 -5.483626721825666e+05 2.1e+00 9.61e-09  7e-09  9e-09 0:20.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -5.483626721825668e+05 2.4e+00 7.03e-09  5e-09  6e-09 0:22.2
 1200  18000 -5.483626721825668e+05 2.6e+00 7.00e-09  5e-09  7e-09 0:24.2
 1300  19500 -5.483626721825668e+05 2.8e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [489874.99918622] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=489874.999186, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=489874.999186, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)


  100   1500 -6.693508001874707e+04 1.6e+00 7.85e-01  7e-01  8e-01 0:02.1
  200   3000 -1.261639430329124e+05 1.8e+00 1.56e-01  1e-01  2e-01 0:04.2
  300   4500 -1.279223231531402e+05 1.9e+00 2.50e-02  2e-02  3e-02 0:06.3
  400   6000 -1.279964367347120e+05 2.8e+00 6.14e-03  4e-03  9e-03 0:08.5
  500   7500 -1.279986159302360e+05 3.1e+00 6.27e-04  4e-04  9e-04 0:10.6
  600   9000 -1.279986594861101e+05 3.4e+00 1.22e-04  8e-05  2e-04 0:12.7
  700  10500 -1.279986617168506e+05 5.1e+00 8.86e-05  5e-05  2e-04 0:14.8
  800  12000 -1.279986640018749e+05 1.1e+01 8.21e-05  5e-05  4e-04 0:17.0
  900  13500 -1.279986642615794e+05 1.2e+01 9.58e-06  5e-06  5e-05 0:19.1
 1000  15000 -1.279986642650426e+05 1.2e+01 7.50e-07  4e-07  4e-06 0:21.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.279986642650605e+05 1.2e+01 5.85e-08  3e-08  3e-07 0:23.3
 1200  18000 -1.279986642650607e+05 1.2e+01 2.96e-08  1e-08  1e-07 0:25.4
 1300  19500 -1.279986642650606e+05 1.3e

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F20_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F20(X_Values.iloc[i,:50])
    SVM_Fun [i] = F20(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F20(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([18348.63685]), count=array([1])) 0.0006527519356203727
SVM
ModeResult(mode=array([287795.51474528]), count=array([1])) 167077.94820930163
ELN
ModeResult(mode=array([17931.04106245]), count=array([1])) 720.600485444447
